# Expected variants
This script determines the expected number of variants for a given region. 
It calculates this for:
- Transcripts
- NMD regions

## Import modules

In [10]:
import numpy as np
import pandas as pd
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
from statsmodels.stats.proportion import proportions_ztest
from statsmodels.stats.multitest import fdrcorrection as fdr
from sklearn.metrics import r2_score
from scipy import stats as _stats

sns.set_context("talk")

## Load data

### Rare synonymous variants are the basis for the model

In [2]:
# Rare synonymous variants per variant context
syn = pd.read_csv("../outputs/observed_variants_stats_synonymous.tsv", sep="\t")

# Get proportion of variants observed
syn["prop_obs"] = syn["obs"] / syn["pos"]

# Exclude CpG transitions
syn_cpg = syn[syn["variant_type"] != "CpG"].copy()

### Transcripts and NMD regions

In [3]:
# Variants observed per transcript
enst = pd.read_csv("../outputs/observed_variants_stats_transcript_no_cpg.tsv", sep="\t")

In [4]:
# Variants observed per NMD region
nmd = pd.read_csv("../outputs/observed_variants_stats_nmd_no_cpg.tsv", sep="\t")

In [5]:
# Concatenate the transcript-level and region-level data
enst = enst.assign(region="transcript")
nmd = nmd.rename(columns={"nmd": "region"})

df = pd.concat([nmd, enst]).sort_values(["region", "enst", "csq"])

## Linear model for expected proportion of variants

From expecation_model_choices.ipynb, it seems the best model for predicting the expected number of variants (excluding CpG transitions) is a simple linear model of obs vs mu, weighted by the number of possible variants per context. 

In [6]:
# Linear model
fit = np.polyfit(syn_cpg["mu"], syn_cpg["prop_obs"], 1, w=syn_cpg["pos"])
lm_p = np.poly1d(fit)

## Calculate expected variants per transcript and context

In [7]:
def get_expected(df, prediction):
    """
    Get the expected proportion of variants per transcript and consequence.
    prediction is a lambda function used to predict the expected value.
    """
    df = (
        df.assign(
            prop_obs=lambda x: x["n_obs"] / x["n_pos"],
            prop_exp=prediction,
            n_exp=lambda x: np.round(x["n_pos"] * x["prop_exp"], 2),
            oe=lambda x: x["n_obs"] / x["n_exp"],
        )
        .set_index(["region", "enst", "csq"])
        .unstack(fill_value=0)
        .stack()
        .reset_index()[
            [
                "region",
                "enst",
                "csq",
                "mu",
                "n_pos",
                "n_obs",
                "n_exp",
                "oe",
                "prop_obs",
                "prop_exp",
            ]
        ]
    )
    # Z scores and p-values
    df["z"] = df.apply(
        lambda x: (
            proportions_ztest(
                x["n_obs"],
                x["n_pos"],
                x["prop_exp"],
                alternative="smaller",
                prop_var=x["prop_exp"],
            )[0]
        ),
        axis=1,
    )
    df["p"] = df.apply(
        lambda x: proportions_ztest(
            x["n_obs"],
            x["n_pos"],
            x["prop_exp"],
            alternative="smaller",
            prop_var=x["prop_exp"],
        )[1],
        axis=1,
    )
    return df

In [8]:
%%capture
# Ignore divide by 0 errors for regions with no variants

df = get_expected(df=df, prediction=lambda x: lm_p(x["mu"]))

## Write to output

In [9]:
df.to_csv(
    "../outputs/expected_variants_stats_all_regions_no_cpg.tsv", sep="\t", index=False
)

In [22]:
dfa = df.copy()

for r in dfa["region"].unique():
    p = dfa.loc[(df.region == r), ["p"]].dropna()
    p["fdr_p"] = fdr(pvals=p["p"])[1]

In [23]:
p.head()

,p,fdr_p
175938,3.530689e-10,4.701698e-09
175939,9.561315e-03,3.172886e-02
175940,2.404568e-01,4.242680e-01
175941,1.389407e-01,2.796119e-01
175942,5.067710e-03,1.850705e-02
